## 1.数据加载及标准化

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine

# 加载数据
wine = load_wine()
# 标准化
scaler =StandardScaler()
X = scaler.fit_transform(wine.data)
y = wine.target

## 2.自定义K-means算法函数

In [5]:
import numpy as np
def euclidean_distance(one_sample, X):
    one_sample = one_sample.reshape(1, -1)
    distances = np.power(np.tile(one_sample, (X.shape[0], 1)) - X, 2).sum(axis=1)
    return distances

In [6]:
class Kmeans():
    '''
    Parameters:
    k:int
        聚类数目
    max_iter:int
        最大迭代次数
    varepsilon:float
        收敛阈值
    '''
    def __init__(self, k=2, max_iter=500, varepsilon=1e-4):
        self.k = k
        self.max_iter = max_iter
        self.varepsilon = varepsilon
        np.random.seed(1)
    #从样本中随机选取k个样本作为初始聚类中心点
    def init_random_centroids(self, X):
        n_samples, n_features = X.shape
        centroids = np.zeros((self.k, n_features))
        for i in range(self.k):
            centroid = X[np.random.choice(range(n_samples))]
            centroids[i] = centroid
        return centroids
    #返回距离该样本最近的一个中心索引[0,k)
    def _closest_centroid(self, sample, centroids):
        distances = euclidean_distance(sample, centroids)
        closest_i = np.argmin(distances)
        return closest_i
    #进行聚类
    def create_clusters(self, centroids, X):
        clusters = [[] for _ in range(self.k)]
        for sample_i, sample in enumerate(X):
            centroid_i = self._closest_centroid(sample, centroids)
            clusters[centroid_i].append(sample_i)
        return clusters
    #更新中心
    def update_centroids(self, clusters, X):
        n_features = X.shape[1]
        centroids = np.zeros((self.k, n_features))
        for i, cluster in enumerate(clusters):
            centroid = np.mean(X[cluster], axis=0)
            centroids[i] = centroid
        return centroids
    #将所有样本进行归类
    def get_cluster_labels(self, clusters, X):
        y_pred = np.zeros(X.shape[0])
        for cluster_i, cluster in enumerate(clusters):
            for sample_i in cluster:
                y_pred[sample_i] = cluster_i
        return y_pred
    #进行聚类，返回标签
    def predict(self, X):
        centroids = self.init_random_centroids(X)
        for _ in range(self.max_iter):
            clusters = self.create_clusters(centroids, X)
            former_centroids = centroids
            #更新
            centroids = self.update_centroids(clusters, X)
            #收敛
            diff = centroids - former_centroids
            if diff.any() < self.varepsilon:
                break
        return self.get_cluster_labels(clusters, X)

## 3.对数据集进行聚类

In [7]:
# 创建聚类模型
km = Kmeans(k=3)
y_pred = km.predict(X)

*由于聚类标签和实际的不同，需标签统一化*

In [8]:
y[y == 0] = -1
y[y == 1] = -2
y[y == 2] = -3
y

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3])

In [9]:
y_pred[y_pred == 0] = -1
y_pred[y_pred == 2] = -2
y_pred[y_pred == 1] = -3
y_pred

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -2., -2., -3., -2., -2., -2.,
       -2., -2., -2., -2., -1., -2., -2., -2., -1., -2., -2., -2., -2.,
       -1., -2., -2., -2., -2., -3., -2., -2., -2., -2., -2., -2., -2.,
       -2., -2., -2., -2., -1., -2., -2., -2., -2., -2., -2., -2., -2.,
       -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
       -2., -3., -2., -2., -1., -2., -2., -2., -2., -2., -2., -2., -2.,
       -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3.,
       -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3.,
       -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3.,
       -3., -3., -3., -3., -3., -3., -3., -3., -3.])

In [10]:
# 计算吻合度
acc = accuracy_score(y, y_pred)
print("聚类的精确度：{:.2f}".format(acc))

聚类的精确度：0.96


## 4.调用sklearn中的KMeans

In [15]:
from sklearn.cluster import KMeans
def kmeans_cluster(data):
    km1 = KMeans(n_clusters=3, random_state=888)
    result = km1.fit_predict(data)

    return result

In [16]:
# 聚类
y_pred = kmeans_cluster(X)
y[y == 0] = -1
y[y == 1] = -2
y[y == 2] = -3
y

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3])

In [17]:
y_pred[y_pred == 2] = -1
y_pred[y_pred == 1] = -2
y_pred[y_pred == 0] = -3
y_pred

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -2, -2, -3, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -1, -2, -2, -2, -2, -2, -2, -2, -2, -2, -3, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -1, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -3,
       -2, -2, -1, -2, -2, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3], dtype=int32)

In [18]:
# 计算吻合度
acc = accuracy_score(y, y_pred)
print("聚类的精确度：{:.2f}".format(acc))

聚类的精确度：0.97
